In [48]:
#Installing requirements
!pip install -q rdflib sparqlwrapper owlrl gdown pandas wikipedia-api openai

In [49]:
#importing libraries
import gdown
import os
import pandas as pd
import rdflib
from SPARQLWrapper import SPARQLWrapper

In [50]:
# create directory for data
if not os.path.exists("./data/"):
    os.mkdir("./data/")

In [52]:
# Download data
gdown.download(id="1UILSMenvqzFMsIhV6l4ZV1opJpRQZnwC", output="./data/rym_top_5000_all_time.csv", quiet=True)
gdown.download(id="1UNJ2iDX-xoWIekSA0ZwoqNOk3eAZjqrp", output="./data/albumlist.csv", quiet=True)

'./data/albumlist.csv'

In [ ]:
df1 = pd.read_csv('data/albumlist.csv', encoding='mac_roman')
df1.head()

,Number,Year,Album,Artist,Genre,Subgenre
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock"
1,2,1966,Pet Sounds,The Beach Boys,Rock,"Pop Rock, Psychedelic Rock"
2,3,1966,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock"
3,4,1965,Highway 61 Revisited,Bob Dylan,Rock,"Folk Rock, Blues Rock"
4,5,1965,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock


In [ ]:
df2 = pd.read_csv('data/rym_top_5000_all_time.csv', encoding='mac_roman')
df2.head()

,Ranking,Album,Artist Name,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews
0,1.0,OK Computer,Radiohead,16 June 1997,"Alternative Rock, Art Rock","melancholic, anxious, futuristic, alienation, ...",4.23,"70,382",1531
1,2.0,Wish You Were Here,Pink Floyd,12 September 1975,"Progressive Rock, Art Rock","melancholic, atmospheric, progressive, male vo...",4.29,"48,662",983
2,3.0,In the Court of the Crimson King,King Crimson,10 October 1969,"Progressive Rock, Art Rock","fantasy, epic, progressive, philosophical, com...",4.30,"44,943",870
3,4.0,Kid A,Radiohead,3 October 2000,"Art Rock, Experimental Rock, Electronic","cold, melancholic, futuristic, atmospheric, an...",4.21,"58,590",734
4,5.0,To Pimp a Butterfly,Kendrick Lamar,15 March 2015,"Conscious Hip Hop, West Coast Hip Hop, Jazz Rap","political, conscious, poetic, protest, concept...",4.27,"44,206",379


In [ ]:
# In a very Brute Force way I check the number of albums common to the two datasets
counter = 0
for album in df1['Album']:
  for check in df2['Album']:
    if album == check: counter += 1

counter

336

In [ ]:
df1.dtypes

Number       int64
Year         int64
Album       object
Artist      object
Genre       object
Subgenre    object
dtype: object

In [ ]:
df2.dtypes

Ranking              float64
Album                 object
Artist Name           object
Release Date          object
Genres                object
Descriptors           object
Average Rating       float64
Number of Ratings     object
Number of Reviews      int64
dtype: object

In [ ]:
df1 = df1.drop(['Number'], axis=1)
df1 = df1.rename(columns = {'Year' : 'Release_Date', 'Genre' : 'Genres'})
df2 = df2.drop(['Ranking'], axis=1)
df2 = df2.rename(columns = {'Artist Name' : 'Artist', 'Release Date' : 'Release_Date'})

# I decided to drop the columns about the ratings due to their specificity in the domain, they are indeed not useful for integrate further data
df2 = df2.drop(['Average Rating', 'Number of Ratings', 'Number of Reviews'], axis=1)

In [ ]:
df2.columns.intersection(df1.columns)

Index(['Album', 'Artist', 'Release_Date', 'Genres'], dtype='object')

Concatanaiting the two dataframes


In [ ]:
final_df = pd.concat([df1, df2])
final_df.columns

Index(['Release_Date', 'Album', 'Artist', 'Genres', 'Subgenre', 'Descriptors'], dtype='object')

In [ ]:
final_df = final_df.drop_duplicates(subset='Album', keep = 'last')
final_df = final_df.reset_index(drop = True)
final_df.head()

,Release_Date,Album,Artist,Genres,Subgenre,Descriptors
0,1968,"The Beatles (""The White Album"")",The Beatles,Rock,"Rock & Roll, Pop Rock, Psychedelic Rock, Exper...",NaN
1,1976,The Sun Sessions,Elvis Presley,Rock,Rock & Roll,NaN
2,1982,The Great Twenty_Eight,Chuck Berry,Rock,Rock & Roll,NaN
3,1990,The Complete Recordings,Robert Johnson,Blues,Delta Blues,NaN
4,1970,John Lennon/Plastic Ono Band,John Lennon / Plastic Ono Band,Rock,Pop Rock,NaN


In [ ]:
final_df.tail()

,Release_Date,Album,Artist,Genres,Subgenre,Descriptors
5096,17 July 1970,Gracious!,Gracious,"Progressive Rock, Symphonic Prog",NaN,NaN
5097,18 May 1998,Knights of the Cross,Grave Digger,"Power Metal, Heavy Metal",NaN,"history, male vocals, heavy, concept album, an..."
5098,12 November 2007,(Listen For) The Rag and Bone Man,And Also the Trees,Art Rock,NaN,"atmospheric, poetic, dark, male vocals, ominou..."
5099,1969,Hymnen f√ºr elektronische und konkrete Kl√§nge,Studio f√ºr elektronische Musik des Westdeutsc...,"Musique concr√®te, Modern Classical, Electronic",NaN,"avant-garde, sampling, dense, complex, scary, ..."
5100,22 May 1975,Adventures in Paradise,Minnie Riperton,"Smooth Soul, Pop Soul",NaN,"lush, female vocals, romantic, uplifting, summ..."


In [ ]:
def getYear(val):
  s = str(val)
  return s[len(s)-4:]

final_df['Release_Date'] = final_df['Release_Date'].apply(getYear)
final_df['Genres'] = final_df['Genres'].astype('string')
final_df['Subgenre'] = final_df['Subgenre'].astype('string')
final_df['Descriptors'] = final_df['Descriptors'].astype('string')

In [ ]:
final_df.dtypes

Release_Date    object
Album           object
Artist          object
Genres          string
Subgenre        string
Descriptors     string
dtype: object

In [ ]:
# Formatting string columns
def divideString(val):
  if type(val) == str:
    subgenres = val.split(',')
    new_subgenres = [subgenres[0].replace(' ', '')]
    if len(subgenres) > 1:
      for i in subgenres[1:]:
        new_subgenres.append(i[1:].replace(' ', ''))
    return new_subgenres
  else: return []

for col in ['Genres', 'Subgenre', 'Descriptors']:
  final_df[col] = final_df[col].apply(divideString)

final_df['Album'] = final_df['Album'].str.replace(' ', '_').str.translate({ord(i): None for i in '"\'()/!'})
final_df['Artist'] = final_df['Artist'].str.replace(' ', '_').str.translate({ord(i): None for i in '"\'()/!'})


final_df.head()

,Release_Date,Album,Artist,Genres,Subgenre,Descriptors
0,1968,The_Beatles_The_White_Album,The_Beatles,[Rock],"[Rock&Roll, PopRock, PsychedelicRock, Experime...",[]
1,1976,The_Sun_Sessions,Elvis_Presley,[Rock],[Rock&Roll],[]
2,1982,The_Great_Twenty_Eight,Chuck_Berry,[Rock],[Rock&Roll],[]
3,1990,The_Complete_Recordings,Robert_Johnson,[Blues],[DeltaBlues],[]
4,1970,John_LennonPlastic_Ono_Band,John_Lennon__Plastic_Ono_Band,[Rock],[PopRock],[]


In [ ]:
final_df.tail()

,Release_Date,Album,Artist,Genres,Subgenre,Descriptors
5096,1970,Gracious,Gracious,"[ProgressiveRock, SymphonicProg]",[],[]
5097,1998,Knights_of_the_Cross,Grave_Digger,"[PowerMetal, HeavyMetal]",[],"[history, malevocals, heavy, conceptalbum, ant..."
5098,2007,Listen_For_The_Rag_and_Bone_Man,And_Also_the_Trees,[ArtRock],[],"[atmospheric, poetic, dark, malevocals, ominou..."
5099,1969,Hymnen_f√ºr_elektronische_und_konkrete_Kl√§nge,Studio_f√ºr_elektronische_Musik_des_Westdeutsc...,"[Musiqueconcr√®te, ModernClassical, Electronic]",[],"[avant-garde, sampling, dense, complex, scary,..."
5100,1975,Adventures_in_Paradise,Minnie_Riperton,"[SmoothSoul, PopSoul]",[],"[lush, femalevocals, romantic, uplifting, summ..."


In [ ]:
final_df.dtypes

Release_Date    object
Album           object
Artist          object
Genres          object
Subgenre        object
Descriptors     object
dtype: object

Starting to create the graph


In [53]:
from rdflib import Graph, URIRef, BNode, Literal, FOAF, RDF, OWL, RDFS, XSD, Namespace

In [11]:
graph = Graph()

# Number of albums to work with, I added this constant in order to make the user able to decide how much data to work with
# obviously keep in mind that the number of triples in the graph grows exponentially with respect to the number of albums
NUMBER_OF_ALBUMS = 1000000

# working at first on example namespace
ex = Namespace('http://example.org/')
dbp = Namespace('https://dbpedia.org/property/')
dbo = Namespace('https://dbpedia.org/ontology/')
dbr = Namespace('https://dbpedia.org/resource/')
graph.namespace_manager.bind('ex', ex)
graph.namespace_manager.bind('dbp', dbp)
graph.namespace_manager.bind('dbo', dbo)
graph.namespace_manager.bind('dbr', dbr)

Adding triples already integrated with DBPedia

In [ ]:
def check_entity_presence(entity_uri):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")

    # Construct the SPARQL query to check for the entity
    query = f"""
        ASK WHERE {{
            {entity_uri} ?p ?o .
        }}
    """

    sparql.setQuery(query)
    sparql.setReturnFormat("json")

    # Execute the query and check the result
    try:
      result = sparql.query().convert()
    except: return False

    return result['boolean']

#for each row of the dataset are added (3 + # of genres + # of subgenres + # of descriptors) triples
def addTriples(graph, album_uri, album, artist, date, genres, subgenres, descriptors):
  graph.add((album_uri, dbp.artist, artist))
  graph.add((album_uri, RDFS.label, Literal(album, lang = 'en')))
  graph.add((album_uri, dbp.released, Literal(date, datatype = XSD.date)))

  genres_uris = []
  for genre in genres:
      if check_entity_presence(f'dbr:{genre}'):
        graph.add((album_uri, dbp.genre, dbr[genre]))
        graph.add((dbr[genre], RDF.type, ex.Genre))
        genres_uris.append((genre, 'dbr'))
      else:
        graph.add((album_uri, dbp.genre, ex[genre]))
        graph.add((ex[genre], RDF.type, ex.Genre))
        genres_uris.append((genre, 'ex'))

  for subgenre in subgenres:
      if check_entity_presence(f'dbr:{subgenre}'):
        graph.add((dbr[subgenre], RDF.type, ex.Subgenre))
        for genre, namespace in genres_uris:
          if namespace == 'dbr': graph.add((dbr[subgenre], dbo.MusicSubgenre, dbr[genre]))
          else: graph.add((dbr[subgenre], dbo.MusicSubgenre, ex[genre]))
      else:
        graph.add((ex[subgenre], RDF.type, ex.Subgenre))
        for genre, namespace in genres_uris:
          if namespace == 'dbr': graph.add((ex[subgenre], dbo.MusicSubgenre, dbr[genre]))
          else: graph.add((ex[subgenre], dbo.MusicSubgenre, ex[genre]))

  for descriptor in descriptors:
    graph.add((album_uri, ex.descriptor, ex[descriptor]))



In [ ]:
artists_dbr = []

for i in range(min([NUMBER_OF_ALBUMS, final_df.shape[0]])):
  row = final_df.loc[i]
  album = row['Album']
  artist = row['Artist']
  date = row['Release_Date']
  genres = row['Genres']
  subgenres = row['Subgenre']
  descriptors = row['Descriptors']

  if check_entity_presence(f'dbr:{album}'):
    album_uri = dbr[album]
  else: album_uri = ex[album]

  if check_entity_presence(f'dbr:{artist}'):
    artists_dbr.append(artist)
    artist = dbr[artist]
  else: artist = ex[artist]

  addTriples(graph, album_uri, album, artist, date, genres, subgenres, descriptors)

In [ ]:
print(len(graph))

67715


In [ ]:
print(len(artists_dbr))

4141


In [ ]:
# Saving the first partial result, the commented line is used to save it also in the drive
graph.serialize(destination="./data/first_triples.ttl")
#graph.serialize(destination="./drive/MyDrive/MusicGraph/first_triples.ttl")

<Graph identifier=N39042d4d8d3f48e8a5333f2cb5cdaa92 (<class 'rdflib.graph.Graph'>)>

Implementing informations about the artists


In [ ]:
# Uncomment this block of code if you want to load the knowledge graph without executing the previous code
# (make sure the file is in the correct folder)
#graph = Graph()
#graph.parse(source = "./data/first_triples.ttl")

In [ ]:
# returns true if the artist is a dbo:Band
def isBand(artist):
  sparql = SPARQLWrapper("http://dbpedia.org/sparql")

  query = f"""
      ASK WHERE {{
          dbr:{artist} rdf:type dbo:Band .
      }}
  """

  sparql.setQuery(query)
  sparql.setReturnFormat("json")

  try:
    result = sparql.query().convert()
  except: return False

  return result['boolean']

#returns the uris of the members of the band
def getMembers(band):
  sparql = SPARQLWrapper("http://dbpedia.org/sparql")

  query = f"""
      SELECT ?x WHERE {{
          dbr:{band} dbo:bandMember|dbo:formerBandMember ?x .
      }}
  """

  sparql.setQuery(query)
  sparql.setReturnFormat("json")

  try:
    result = sparql.query().convert()
  except: return []

  return [i['x']['value'] for i in result['results']['bindings']]

# returns the birthplace and the coutry of birth of an artist
def getBirthPlace(bandMember):
  sparql = SPARQLWrapper("http://dbpedia.org/sparql")

  query = f"""
      SELECT ?x ?y WHERE {{
          dbr:{bandMember} dbo:birthPlace ?x .
          OPTIONAL {{ ?x dbo:country ?y . }}
      }}
  """

  sparql.setQuery(query)
  sparql.setReturnFormat("csv")

  try:
    result = sparql.query().convert()
  except:
    return []

  return result.decode()

# returns the founding city and the founding country of the band
def getFoundingCity(band):
  sparql = SPARQLWrapper("http://dbpedia.org/sparql")

  query = f"""
      SELECT DISTINCT ?x ?y WHERE {{
          dbr:{band} dbo:hometown ?x .
          OPTIONAL {{ ?x dbo:country ?y .}}
      }}
  """

  sparql.setQuery(query)
  sparql.setReturnFormat("json")

  try:
    result = sparql.query().convert()['results']['bindings'][0]
  except:
    return []

  return [result[i]['value'] for i in result]

In [ ]:
# remove duplicate artists
artists_unique = set(artists_dbr)
print(f'{len(artists_unique)} unique artists out of all the artists saved')

2104 unique artists out of all the artists saved


In [ ]:
# Iterating through the artists found in DBPedia and looking for the bands in order to add information about them
for artist in artists_unique:
  if isBand(artist):
    founding = getFoundingCity(artist)
    graph.add((dbr[artist], RDF.type, dbo.Band))
    if len(founding) == 2:
      graph.add((dbr[artist], ex.foundingCity, URIRef(founding[0])))
      graph.add((dbr[artist], ex.foundingCountry, URIRef(founding[1])))
    elif len(founding) == 1:
      graph.add((dbr[artist], ex.foundingCity, URIRef(founding[0])))
    members = getMembers(artist)
    if len(members) > 0:
        for member in members:
          member_uri = URIRef(member)
          birthPlace = getBirthPlace(member)
          graph.add((dbr[artist], dbo.bandMember, member_uri))
          if len(birthPlace) == 2:
            graph.add((member_uri, dbo.birthPlace, URIRef(birthPlace[0])))
            graph.add((member_uri, ex.birthCountry, URIRef(birthPlace[1])))
          elif len(birthPlace) == 1:
            graph.add((member_uri, dbo.birthPlace, URIRef(birthPlace[0])))


In [ ]:
print(len(graph))

72778


In [ ]:
# Saving the second partial result, the commented line is used to save it also in the drive
graph.serialize(destination="./data/second_stage_triples.ttl")
#graph.serialize(destination="./drive/MyDrive/MusicGraph/triples_with_members.ttl")

<Graph identifier=N39042d4d8d3f48e8a5333f2cb5cdaa92 (<class 'rdflib.graph.Graph'>)>

Integrating wikipedia's data

In [ ]:
# Uncomment this block of code if you want to load the knowledge graph without executing the previous code
# (make sure the file is in the correct foldet)
#graph = Graph()
#graph.parse(source = "./data/triples_with_members.ttl")

In [ ]:
import wikipediaapi

In [ ]:
# returns the wikipedia url of artist_uri
def queryWikiPage(artist_uri):
  sparql = SPARQLWrapper("http://dbpedia.org/sparql")
  query = f"""
      SELECT ?url WHERE {{
          dbr:{artist_uri.split('/')[-1]} foaf:isPrimaryTopicOf ?url
      }}
  """
  sparql.setQuery(query)
  sparql.setReturnFormat("json")
  try:
    result = sparql.query().convert()['results']['bindings'][0]['url']['value']
  except:
    query = f"""
      SELECT ?url WHERE {{
          dbr:{artist_uri.split('/')[-1]} foaf:primaryTopic ?url
      }}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat("json")
    try:
      result = sparql.query().convert()['results']['bindings'][0]['url']['value']
    except:
      return None
  return result

In [ ]:
# Not working due to openai billing policy
import openai
openai_key = ''
openai.api_key = openai_key

def getRoles(artist_uri):
  artist_name = str(artist_uri)[:-4].split('/')[-1]
  wiki_wiki = wikipediaapi.Wikipedia('MusicGraph by Francesco_Casone', 'en')
  summary = ' '.join(wiki_wiki.page(artist_name).summary.split('.')[:2])

  response = openai.Completion.create(
    model="gpt-3.5-turbo-0613",
    prompt=f"{summary} .Given this paragraph, what is the role of {artist_name} in his band? Express the answer as a list of comma separated single words"
  )
  return response["choices"][0]["message"]

# return the keywords found in artist_uri's summary of wikipedia
def getRoleByKeywords(artist_uri):
  wiki_api = wikipediaapi.Wikipedia('MusicGraph by Francesco_Casone', 'en')
  artist_name = str(artist_uri)[:-4].split('/')[-1]
  keywords = ['drummer', 'singer', 'guitarist', 'musician', 'disc jockey', 'dj', 'producer', 'songwriter']
  summary = ''

  if len(wiki_api.page(artist_name).summary) > 0:
    summary = wiki_api.page(artist_name).summary
  else:
    wiki_page = queryWikiPage(artist_uri)
    print(wiki_page)
    if wiki_page:
      wiki_page = wiki_page.split('/')[-1]
      summary = wiki_api.page(wiki_page).summary

  if len(summary) > 0:
    for word in keywords:
      if word in summary.lower():
        graph.add((artist_uri, ex.hasRole, ex[word.replace(' ', '_')]))


In [ ]:
#get all the band members in the graph
def getAllMembers():
  query = f"""
      SELECT DISTINCT ?member WHERE {{
          {{?x a dbo:Band .
          ?x dbo:bandMember ?member .}}
            UNION
          {{?x a dbo:Band .
          ?x dbo:formerBandMember ?member .}}
      }}
  """
  result = [row.member for row in graph.query(query)]
  return result

In [ ]:
# Iterating through all the band members in the graph to add the keywords found  in wikipedia about them
members = getAllMembers()
print(len(members))
for member in members:
  try:
    getRoleByKeywords(member)
  except: pass

In [ ]:
len(graph)

76365

In [21]:
# Saving the third partial result, the commented line is used to save it also in the drive
graph.serialize(destination="./data/graph_with_roles.ttl")
#graph.serialize(destination="./drive/MyDrive/MusicGraph/graph_with_roles.ttl")

<Graph identifier=N53ddcce1dd57456bb645615461ec081a (<class 'rdflib.graph.Graph'>)>

Ontology


In [23]:
# adding classes using the example domain
graph.add((ex.Genre, RDF.type, OWL.Class))
graph.add((ex.SubGenre, RDF.type, OWL.Class))
graph.add((ex.Descriptor, RDF.type, OWL.Class))
graph.add((ex.Artist, RDF.type, OWL.Class))
graph.add((ex.MusicAlbum, RDF.type, OWL.Class))
graph.add((ex.Person, RDF.type, OWL.Class))
graph.add((ex.Band, RDF.type, OWL.Class))
graph.add((ex.City, RDF.type, OWL.Class))
graph.add((ex.Country, RDF.type, OWL.Class))
graph.add((ex.BandRole, RDF.type, OWL.Class))
graph.add((ex.BandMember, RDF.type, OWL.Class))


graph.add((ex.Band, RDFS.subClassOf, ex.Artist))
graph.add((ex.BandMember, RDFS.subClassOf, ex.Artist))
graph.add((ex.Artist, RDFS.subClassOf, ex.Person))
graph.add((ex.Subgenre, RDFS.subClassOf, ex.Genre))

# adding domains and ranges for every property
graph.add((dbp.artist, RDFS.domain, ex.MuisicAlbum))
graph.add((dbp.artist, RDFS.range, ex.Artist))

graph.add((dbp.released, RDFS.domain, ex.MuisicAlbum))

graph.add((dbp.genre, RDFS.domain, ex.MuisicAlbum))
graph.add((dbp.genre, RDFS.range, ex.Genre))

graph.add((dbo.MusicSubgenre, RDFS.domain, ex.Subgenre))
graph.add((dbo.MusicSubgenre, RDFS.range, ex.Genre))

graph.add((ex.descriptor, RDFS.domain, ex.MuisicAlbum))
graph.add((ex.descriptor, RDFS.range, ex.Descriptor))

graph.add((ex.foundingCity, RDFS.domain, ex.Band))
graph.add((ex.foundingCity, RDFS.range, ex.City))

graph.add((ex.foundingCountry, RDFS.domain, ex.Band))
graph.add((ex.foundingCountry, RDFS.range, ex.Country))

graph.add((dbo.bandMember, RDFS.domain, ex.Band))
graph.add((dbo.bandMember, RDFS.range, ex.BandMember))

graph.add((ex.hasRole, RDFS.domain, ex.BandMember))
graph.add((ex.hasRole, RDFS.range, ex.BandRole))

graph.add((dbo.birthPlace, RDFS.domain, ex.Artist))
graph.add((dbo.birthplace, RDFS.range, ex.City))

graph.add((dbo.birthCountry, RDFS.domain, ex.Artist))
graph.add((dbo.birthCountry, RDFS.range, ex.Country))


<Graph identifier=N53ddcce1dd57456bb645615461ec081a (<class 'rdflib.graph.Graph'>)>

In [24]:
len(graph)

76401

In [25]:
# Materializing the inferences produced by the ontology, it is currently using the RDFS semantics
import owlrl
owlrl.DeductiveClosure(owlrl.RDFS_Semantics).expand(graph)
print(len(graph))

114140


In [27]:
# Saving the final result, the commented line is used to save it also in the drive
graph.serialize(destination="./data/graph_after_closure.ttl")
#graph.serialize(destination="./drive/MyDrive/MusicGraph/graph_after_closure.ttl")

<Graph identifier=N53ddcce1dd57456bb645615461ec081a (<class 'rdflib.graph.Graph'>)>

Querying the graph

In [54]:
# Uncomment this block of code if you want to load the graph without executing the previous code
# (make sure the file is in the correct folder)
#graph = Graph()
#graph.parse(source = "./data/graph_after_closure.ttl")

<Graph identifier=N331248ed767643bd803a8d62d867a32e (<class 'rdflib.graph.Graph'>)>

In [ ]:
# Looking for all the countries present in the dataset
query = f"""
    SELECT ?country WHERE {{
        ?country a  ex:Country
    }}
"""

result = graph.query(query)
countries = [row.country.split('/')[-1] for row in result]
countries_series = pd.Series(countries)
countries_series  # wanna add the number of bands for each country

0                 Austria
1                 Hungary
2                  Israel
3             New_Zealand
4                  Russia
5                Scotland
6                 Senegal
7                  Serbia
8       South_Lanarkshire
9             Switzerland
10                Denmark
11                Jamaica
12                 Mexico
13                  Spain
14                 Canada
15                 Greece
16            Netherlands
17               Portugal
18                  Chile
19    Republic_of_Ireland
20              Argentina
21                 Brazil
22                 Norway
23                 France
24                Germany
25                 Poland
26                Germany
27                  Japan
28              Australia
29                  Italy
30                Finland
31                 Sweden
32         United_Kingdom
33          United_States
dtype: object

In [ ]:
# Finding the years with the most albums in the dataset
query = f"""
    SELECT ?year WHERE {{
        ?album dbp:released ?year
    }}
"""

result = graph.query(query)
years = [row.year[:4] for row in result]
years_set = list(set(years))
count = []
for year in years_set:
  count.append(years.count(year))
years_count_df = pd.DataFrame({ 'Year' : years_set , 'Number of albums' : count })
years_count_df.sort_values('Number of albums', ascending = False).reset_index(drop = True)

,Year,Number of albums
0,1971,160
1,1973,159
2,1972,153
3,1970,148
4,1994,140
...,...,...
67,1954,6
68,1953,2
69,1952,1
70,1947,1


In [ ]:
# Looking for the genres that appear togheter more often
query = f"""
    SELECT ?first_genre ?second_genre WHERE {{
        ?album dbp:genre ?first_genre .
        ?album dbp:genre ?second_genre .
        FILTER (?first_genre != ?second_genre) .
    }}
"""

result = graph.query(query)
couples = [(row.first_genre.split('/')[-1], row.second_genre.split('/')[-1]) for row in result]
couples_set = list(set(couples))
couples_count = []
for couple in couples_set:
  couples_count.append(couples.count(couple))
coupled_genres_df = pd.DataFrame({ 'First Genre' : [couple[0] for couple in couples_set], 'Second Genre' : [couple[1] for couple in couples_set] , 'Number of albums' : couples_count })
coupled_genres_df.sort_values('Number of albums', ascending = False).reset_index(drop = True)

,First Genre,Second Genre,Number of albums
0,SymphonicProg,ProgressiveRock,72
1,ProgressiveRock,SymphonicProg,72
2,BoomBap,EastCoastHipHop,66
3,EastCoastHipHop,BoomBap,66
4,Songwriter,ContemporaryFolk,61
...,...,...,...
3775,SpaceAmbient,Soundtracks,1
3776,IndieRock,MathRock,1
3777,WesternClassicalMusic,ChamberMusic,1
3778,Plunderphonics,Nu-Disco,1


In [ ]:
# Looking for the most influential city, the city that produces most talents in music
query = f"""
    SELECT ?city WHERE {{
        ?band ex:foundingCity ?city
    }}
"""
result = graph.query(query)
cities = [row.city.split('/')[-1].replace('_', ' ') for row in result]
cities_set = list(set(cities))
count = []
for city in cities_set:
  count.append(cities.count(city))
cities_count_df = pd.DataFrame({ 'City' : cities_set , 'Number of artists' : count })
cities_count_df.sort_values('Number of artists', ascending = False).reset_index(drop = True)

,City,Number of artists
0,London,37
1,California,33
2,New York City,10
3,Birmingham,9
4,Washington (state),8
...,...,...
353,"Brooklyn, New York",1
354,Derry,1
355,East Kilbride,1
356,Minnesota,1


In [47]:
# Looking for the most commmon genre in each country
query = f"""
    SELECT ?country ?genre WHERE {{
        ?album dbp:genre ?genre .
        ?album dbp:artist [a ex:Band; ex:foundingCountry ?country]
    }}
"""
result = list(graph.query(query))
res = {}
for i in result:
  country = i[0].split('/')[-1]
  genre = i[1].split('/')[-1]
  if country in res.keys():
    res[country].append(genre)
  else:
    res[country] = [genre]

couples = []
for country in res:
  most_common = max(set(res[country]), key=res[country].count)
  couples.append((country, most_common))

res = pd.DataFrame({'Country' : [i[0] for i in couples], 'Most Common Genre' : [i[1] for i in couples]})
res.head()

,Country,Most Common Genre
0,Italy,ProgressiveRock
1,Jamaica,RootsReggae
2,United_States,EastCoastHipHop
3,United_Kingdom,ProgressiveRock
4,Sweden,MelodicDeathMetal


Personal notes:
- I thought about adding every entity to the example domain and adding the triple (ex:entity, owl:sameAs, dbr:entity) for every entity that the program could find in the dbpedia graph. This would have led to a complete personal domain linked to the dbpedia graph (I still think that this is good and easy thing to implement), which I imagine can be useful in some scenarios. But for the sake of time and space expenses I just added to the graph the entity in the dbpedia domain where those could be found and the entity in the example domain in every other case.
- I implemented the classes as OWL:Class even if RDFS:Class would be enough just for the purpose of scalability. I want indeed to scale it up adding more classes and more complex properties.
